In [ ]:
import numpy as np
import PIL
import cv2

from sklearn.metrics import confusion_matrix
from keras.optimizers import Adam
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# fonte: https://gist.github.com/jovianlin/fd38110bffad9158defd8db1a3309ee3

def plot_images(images, img_shape, cls_true, cls_pred=None):
    assert len(images) == len(cls_true) == 9
    
    # Create figure with 3x3 sub-plots.
    fig, axes = plt.subplots(3, 3)
    fig.subplots_adjust(hspace=0.3, wspace=0.3)

    for i, ax in enumerate(axes.flat):
        # Plot image.
        ax.imshow(images[i].reshape(img_shape), cmap='binary')

        # Show true and predicted classes.
        if cls_pred is None:
            xlabel = "True: {0}".format(cls_true[i])
        else:
            xlabel = "True: {0}, Pred: {1}".format(cls_true[i], cls_pred[i])

        # Show the classes as the label on the x-axis.
        ax.set_xlabel(xlabel)
        
        # Remove ticks from the plot.
        ax.set_xticks([])
        ax.set_yticks([])
    
    plt.show()

In [ ]:
# fonte: http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
def execute_model(model, X_train, Y_train, X_test, Y_test, 
                  validation_split, batch_size, cm_plot_labels, 
                  adam_lr, nb_epochs):    
    model.compile(Adam(lr=adam_lr), loss='categorical_crossentropy', metrics=['accuracy'])
    hist = model.fit(X_train,
                     Y_train,
                     validation_split=validation_split, 
                     batch_size=batch_size, 
                     epochs=nb_epochs, 
                     shuffle=True, 
                     verbose=1)

    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title('model train vs validation loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper right')
    plt.show()
    
    plt.plot(hist.history['acc'])
    plt.plot(hist.history['val_acc'])
    plt.title('model train vs validation acc')
    plt.ylabel('acc')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper right')
    plt.show()    
    
    test_labels = np.asarray(get_labels(np.argmax(Y_test, axis=1)))
    predictions = model.predict(X_test)
    pred_labels = np.asarray(get_labels(np.argmax(predictions, axis=1)))
    cm = confusion_matrix(test_labels, pred_labels)
    acertos = np.sum(np.diag(cm))
    print('Acertos:', acertos, '/', Y_test.shape[0])
    plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')
    return acertos, predictions

In [ ]:
def resize_npy_image(np_image, new_size=(128,128)):
    return cv2.resize(np_image, new_size, interpolation=cv2.INTER_CUBIC)

def resize_npy_images(np_images, new_size=(128,128)):
    return [resize_npy_image(np_image, new_size) for np_image in np_images]

In [ ]:
def convert_to_rgb_npy_image(np_image):
    return np.array(PIL.Image.fromarray(np_image.astype('uint8')).convert("RGB"))

def convert_to_rgb_npy_images(np_images):
    return [convert_to_rgb_npy_image(np_image) for np_image in np_images]